![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-15</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [116]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit



In [117]:
#Reading and exploring dataset
df = pd.read_csv('car_insurance.csv')
print(df.head())
print(df.info())
print(df.describe())

       id  age  gender  ... duis past_accidents outcome
0  569520    3       0  ...    0              0     0.0
1  750365    0       1  ...    0              0     1.0
2  199901    0       0  ...    0              0     0.0
3  478866    0       1  ...    0              0     0.0
4  731664    1       1  ...    0              1     1.0

[5 rows x 18 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 n

In [118]:
#Filling null values
df['credit_score'] = df['credit_score'].fillna(df['credit_score'].mean())
df['annual_mileage'] = df['annual_mileage'].fillna(df['annual_mileage'].mean())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         10000 non-null  float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       10000 non-null  float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [119]:
#Preparing for modeling
models=[]
features = df.drop(columns=['outcome', 'id'])
print(features.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  10000 non-null  int64  
 1   gender               10000 non-null  int64  
 2   driving_experience   10000 non-null  object 
 3   education            10000 non-null  object 
 4   income               10000 non-null  object 
 5   credit_score         10000 non-null  float64
 6   vehicle_ownership    10000 non-null  float64
 7   vehicle_year         10000 non-null  object 
 8   married              10000 non-null  float64
 9   children             10000 non-null  float64
 10  postal_code          10000 non-null  int64  
 11  annual_mileage       10000 non-null  float64
 12  vehicle_type         10000 non-null  object 
 13  speeding_violations  10000 non-null  int64  
 14  duis                 10000 non-null  int64  
 15  past_accidents       10000 non-null  

In [120]:
#Building and storing the models
for col in features:
    model = logit(f'outcome~{col}', data=df).fit()
    print(models.append(model))

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
None
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
None
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
None
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
None
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
None
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
None
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
None
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
None
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
None
Optimization terminated successfully.

In [121]:
#Measuring performance
accuracies=[]
for i in range(len(models)):
    conf_matrix = models[i].pred_table()
    true_positives = conf_matrix[0,0]
    true_negatives = conf_matrix[1,1]
    total = sum(sum(row) for row in conf_matrix)
    accuracy = (true_positives + true_negatives) / total
    accuracies.append(accuracy)
    print(accuracies)

[0.7747]
[0.7747, 0.6867]
[0.7747, 0.6867, 0.7771]
[0.7747, 0.6867, 0.7771, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867, 0.6904]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867, 0.6904, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867, 0.6904, 0.6867, 0.6867]
[0.7747, 0.6867, 0.7771, 0.6867, 0.7425, 0.7054, 0.7351, 0.6867, 0.6867, 0.6867, 0.6867, 0.6904, 0.6867, 0.6867, 0.6867]
[0.7747, 0.6867, 0.7771, 

In [122]:
#Finding the best performing model
max_accuracy_index = accuracies.index(max(accuracies))
print(max_accuracy_index)
print(max(accuracies))


2
0.7771


In [123]:
#Mapping the highest accuracy to the feature
best_feature = {'best_feature': features.columns[max_accuracy_index], 'best_accuracy': accuracies[2]}
best_feature_df = pd.DataFrame(data=best_feature, index=[0])
print(best_feature_df)

         best_feature  best_accuracy
0  driving_experience         0.7771
